# Setup

## Github Setup
If not already present this clones the repo into google colab so that I may work on it with a bash environment.

In [0]:
from getpass import getpass
import os

# if not in a repo clone the desired one
if not os.path.isdir("/content/repo"):
    
    # get username and password
    user = "Boyne272"
    password = getpass('github password')
    os.environ['GITHUB_AUTH'] = user + ':' + password
    
    # clone the repo
    !git clone --quiet https://$GITHUB_AUTH@github.com/msc-acse/acse-9-independent-research-project-Boyne272.git repo

    # move into the repo
    %cd repo
    
    # swap to the wanted branch
    !git checkout Release
    !git pull origin Release --quiet

# set name and email
!git config --global user.name "Richard Boyne"
!git config --global user.email "boynerichard@yahoo.co.uk"

# remove the sample data if there
if os.path.isdir("/content/sample_data"):
    !rm -r /content/sample_data
    
# show where we are
!git show --summary

## Github Space
Use the cells here to run github commands

In [0]:
%cd /content/repo

In [0]:
# !git pull origin master
# !git pull origin merging
# !git checkout merging
# !git pull post_processing
# !git branch

In [0]:
!rm -r images/Results

In [0]:
!mv images/Dumb_TS_meme.jpg images/meme.jpg 

In [0]:
!git status

In [0]:
!git rm -r  images/Results/

In [0]:
!git commit -m "uploaded results notebook and removed old results images"

In [0]:
!git push origin Release

## Run Tests
Run all pytests here, try to do as often as possible

In [0]:
!pytest # last run 28th Aug

## Imports

In [0]:
# Ipython magic functions
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# imports
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

# custom imports
%cd /content/repo
from TSA.merging import AGNES
from TSA.merging import segment_group
from TSA.pre_post_processing import Segment_Analyser
from TSA.pre_post_processing import Image_processor
from TSA.kmeans import SLIC
from TSA.kmeans import MSLIC_wrapper

In [0]:
# set matplotlib preferences - report
import matplotlib as mpl
mpl.rcdefaults()
plt.rc('axes', titlesize=30, labelsize=25)
plt.rc('axes.formatter', limits=[-4, 4])
plt.rc('ytick', labelsize=12)
plt.rc('xtick', labelsize=12)
plt.rc('lines', linewidth=2, markersize=7)
plt.rc('figure', figsize=(9, 9))
# print(plt.rcParams) # all parameters

# Results

## showing unblured and blured results

In [0]:
butterfly_IP = Image_processor(path='/content/images/butterfly.tif')
butterfly_IP.plot()

In [0]:
# blur the image to reduce spotting effect
butterfly_IP.reset()
blur = butterfly_IP.gauss(sigma=3, key='blur')

In [0]:
# create the SLIC objects iterate it and plot
blured_SLIC = SLIC(blur, [25, 25], dist_metric_args=[1.])
blured_SLIC.iterate(10)

unblured_SLIC = SLIC(butterfly_IP.imgs['original'], [25, 25], dist_metric_args=[1.])
unblured_SLIC.iterate(10)

In [0]:
fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
blured_SLIC.plot(option='edges', ax=ax)
plt.savefig(r'blured.tif')

fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
unblured_SLIC.plot(option='edges', ax=ax)
plt.savefig(r'unblured.tif')

## Showing different bin grids

In [0]:
big_SLIC = SLIC(butterfly_IP.imgs['blur'], [20, 20], dist_metric_args=[1.])
big_SLIC.iterate(10)

small_SLIC = SLIC(butterfly_IP.imgs['blur'], [40, 40], dist_metric_args=[1.])
small_SLIC.iterate(10)

In [0]:
fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
small_SLIC.plot(option='edges', ax=ax)
ax.set(title='')
ax.axis('off')
plt.savefig(r'high_res.tif')

fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
big_SLIC.plot(option='edges', ax=ax)
ax.set(title='')
ax.axis('off')
plt.savefig(r'low_res.tif')

## Color space comparison


In [0]:
space_SLIC = SLIC(butterfly_IP.imgs['blur'], [25, 25], dist_metric_args=[2.])
space_SLIC.iterate(10)

color_SLIC = SLIC(butterfly_IP.imgs['blur'], [25, 25], dist_metric_args=[.5])
color_SLIC.iterate(10)

In [0]:
fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
space_SLIC.plot(option='edges', ax=ax)
ax.set(title='')
ax.axis('off')
plt.savefig(r'space_bias.tif')

fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
color_SLIC.plot(option='edges', ax=ax)
ax.set(title='')
ax.axis('off')
plt.savefig(r'color_bias.tif')

## SLIC and MSLIC Comparison

In [0]:
IP_example_white = Image_processor(path='images/example_white.tif')
IP_example_polar = Image_processor(path='images/example_polar.tif')

In [0]:
# blur the image to reduce spotting effect
IP_example_white.reset()
IP_example_white.gauss(sigma=3, key='blur')

# blur the image to reduce spotting effect
IP_example_polar.reset()
IP_example_polar.gauss(sigma=3, key='blur')

In [0]:
# SLIC on both
white_SLIC = SLIC(IP_example_white.imgs['blur'], [25, 25], dist_metric_args=[1.])
white_SLIC.iterate(10)

polar_SLIC = SLIC(IP_example_polar.imgs['blur'], [25, 25], dist_metric_args=[1.])
polar_SLIC.iterate(10)

In [0]:
fig, ax = plt.subplots(figsize=[30, 30])
IP_example_white.plot('original', ax=ax)
white_SLIC.plot(option='edges', ax=ax)
ax.set(title='')
ax.axis('off')
plt.savefig(r'white_SLIC.tif')

fig, ax = plt.subplots(figsize=[30, 30])
IP_example_polar.plot('original', ax=ax)
polar_SLIC.plot(option='edges', ax=ax)
ax.set(title='')
ax.axis('off')
plt.savefig(r'polar_SLIC.tif')

In [0]:
# MSLIC with both
example_MSLIC = MSLIC_wrapper([IP_example_white.imgs['blur'],
                               IP_example_polar.imgs['blur']][::-1],
                              [25, 25],
                              dist_metric_args=[1.])
example_MSLIC.iterate(10)

In [0]:
fig, ax = plt.subplots(figsize=[30, 30])
IP_example_white.plot('original', ax=ax)
example_MSLIC.plot([0], option='edges', axs=[ax])
ax.set(title='')
ax.axis('off')
plt.savefig(r'white_MSLIC.tif')

fig, ax = plt.subplots(figsize=[30, 30])
IP_example_polar.plot('original', ax=ax)
example_MSLIC.plot([1], option='edges', axs=[ax])
ax.set(title='')
ax.axis('off')
plt.savefig(r'polar_MSLIC.tif')

In [0]:
###################################
fig, ax = plt.subplots(figsize=[30, 30])
IP_example_white.plot('original', ax=ax)
example_MSLIC.plot([0], option='edges', axs=[ax])
ax.set(title='')
ax.axis('off')
plt.savefig(r'white_MSLIC.tif')

fig, ax = plt.subplots(figsize=[30, 30])
IP_example_polar.plot('original', ax=ax)
example_MSLIC.plot([1], option='edges', axs=[ax])
ax.set(title='')
ax.axis('off')
plt.savefig(r'polar_MSLIC.tif')

## Number of Itertaions

In [0]:
# load the butterfly
butterfly_IP = Image_processor(path='/content/images/butterfly.tif')

# blur the image to reduce spotting effect
butterfly_IP.reset()
butterfly_IP.gauss(sigma=3, key='blur')

# create slic obj
butterfly_SLIC = SLIC(butterfly_IP.imgs['blur'], [25, 25], dist_metric_args=[1.])

In [0]:
butterfly_SLIC.iterate(10)

fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
butterfly_SLIC.plot(option='edges', ax=ax)
plt.savefig(r'iterations_10.tif')

In [0]:
butterfly_SLIC.iterate(10)

fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
butterfly_SLIC.plot(option='edges', ax=ax)
plt.savefig(r'iterations_20.tif')

In [0]:
butterfly_SLIC.iterate(10)

fig, ax = plt.subplots(figsize=[30, 30])
butterfly_IP.plot('original', ax=ax)
butterfly_SLIC.plot(option='edges', ax=ax)
plt.savefig(r'iterations_30.tif')

## Demonstraiting edge detection

In [0]:
# load image
butterfly_IP = Image_processor(path='images/butterfly.tif')

# blur the image to reduce spotting effect
butterfly_IP.reset()
butterfly_IP.gauss(sigma=3, key='blur')

# create the SLIC objects iterate it and plot
butterfly_SLIC = SLIC(butterfly_IP.imgs['blur'], [25, 25], dist_metric_args=[1.])
butterfly_SLIC.iterate(10)

In [0]:
# create the segments
butterfly_mask = butterfly_SLIC.get_segmentation()
butterfly_segs = segment_group(butterfly_mask)
butterfly_segs.enforce_size()

In [0]:
butterfly_segs.plot(back_img=butterfly_IP.imgs['original'])

In [0]:
# use scharr edge detection on original image
fig, ax = plt.subplots(figsize=[15, 15])
butterfly_IP.reset()
butterfly_IP.scharr()
butterfly_IP.plot(ax=ax)
ax.set(title='Three Channel Scharr Edge Detection')
ax.axis('off')
plt.savefig('scharr.tif')

In [0]:
# threshold this image
fig, ax = plt.subplots(figsize=[17, 17])
butterfly_IP.grey_scale()
butterfly_IP.threshold(0.05, key='scharr')
butterfly_IP.plot('scharr', ax=ax)
ax.set(title=r'Greyscale Theshold Image ($Threshold=0.05$)')
ax.axis('off')
plt.savefig('/content/repo/images/Results/binary_scharr.tif')

In [0]:
# define extraction function
def edge_extraction(Xs, Ys, scharr_img):
    return scharr_img[Ys, Xs].mean() / scharr_img.std()

# pass this to the group object and plot it
butterfly_segs.edge_confidence(edge_extraction, [butterfly_IP.imgs['scharr']])

In [0]:
# make the image
fig, ax = plt.subplots(figsize=[16, 16])
butterfly_IP.reset()
ax.imshow(butterfly_IP.grey_scale(), cmap='gray')
butterfly_segs.plot('edge_conf', ax=ax)
ax.axis('off')
plt.savefig('edge_confidence.tif')

In [0]:
# merge by edge confidence
butterfly_segs.merge_by_edge(edge_absent=.5)

# plot the result
fig, ax = plt.subplots(figsize=[16, 16])
butterfly_segs.plot('merged_edges', ax=ax, back_img=butterfly_IP.imgs['original'])
ax.axis('off')
ax.set(title=r'Edge Confidence Merging ($Threshold=0.5$)')
plt.savefig('images/Results/edge_conf_merged.tif')

## AGNES Clustering Figures

In [0]:
# generate dummy 2d data
np.random.seed(10)
features = np.random.normal(size=[500, 2])
features[50:, :] += 5
features[100:, :] += 5
features[150:, 0] += 5
features[250:300, 1] += 10
features[350:400, 0] += 10
features[-1, -1] += 20

plt.figure(figsize=[5, 5])
plt.scatter(*features.T)
plt.title("dummy data")

In [0]:
# iterate the clustering
obj = AGNES(features)
obj.iterate()

In [0]:
fig, axs = plt.subplots(1, 3, figsize=[27, 8])


# ------------------------------------

obj.cluster_distance_plot('dists', last_few=False, ax=axs[0])
axs[0].set(xlabel='Iteration', ylabel='Distance', title='Dummy Data Join Distances')
# plt.savefig('AGNES_distances.tif')


# ------------------------------------


obj.cluster_distance_plot('2nd', last_few=True, ax=axs[1])
axs[1].set(xlabel='Iteration', ylabel='Distance')

std =  0.4297
axs[1].hlines(std, 477, 487, 'r', label=r'$\sigma$')
axs[1].hlines(2*std, 477, 487, 'g', label=r'$2\sigma$')
axs[1].hlines(3*std, 477, 487, 'm', label=r'$3\sigma$')
axs[1].hlines(4*std, 477, 487, 'k', label=r'$4\sigma$')
axs[1].hlines(5*std, 477, 487, 'c', label=r'$5\sigma$')
labs = [r'2nd Derivative', r'+$\sigma$'] + [r'+$%i\sigma$'%i for i in range(2, 6)]
axs[1].legend(labels=labs, loc='upper left', fontsize=15)
axs[1].set(title='Dummy Data Second Derivative')
# plt.savefig('AGNES_derivative.tif')


# ------------------------------------

clusters = obj.cluster_by_derivative(n_std=1., plot=False)

for clust in np.unique(clusters):
    f = features[clusters==clust]
    axs[2].scatter(*f .T, label=clust)

# produce a legend with the unique colors from the scatter
axs[2].legend(loc="upper left", fontsize=15)
axs[2].set(title='Dummy Data AGNES Clustering', xlabel='X Distance',
              ylabel='Y Distance')

plt.savefig('AGNES_clustering.tif')

In [0]:
ax = axs[2]
sca

In [0]:
# fig, ax = plt.subplots(figsize=[10, 10])
# obj.cluster_distance_plot('2nd', last_few=True, ax=ax)
# ax.set(xlabel='Iteration', ylabel='Distance')

# std =  0.4297
# ax.hlines(std, 477, 487, 'r', label=r'$\sigma$')
# ax.hlines(2*std, 477, 487, 'g', label=r'$2\sigma$')
# ax.hlines(3*std, 477, 487, 'm', label=r'$3\sigma$')
# ax.hlines(4*std, 477, 487, 'k', label=r'$4\sigma$')
# ax.hlines(5*std, 477, 487, 'c', label=r'$5\sigma$')
# labs = [r'2nd Derivative', r'+$\sigma$'] + [r'+$%i\sigma$'%i for i in range(2, 6)]
# ax.legend(labels=labs, loc='upper left', fontsize=12)
# ax.set(title='Dummy Data Second Derivative')
# plt.savefig('AGNES_derivative.tif')

In [0]:
# obj.cluster_by_derivative(n_std=1., plot=True)
# plt.gca().set(title='Dummy Data AGNES Clustering', xlabel='X Distance',
#               ylabel='Y Distance')
# plt.savefig('AGNES_clustering.tif')

## Complexity Analysis

In [0]:
# load a large image made by putting 4 smaller images together
paths = ['images/California/white_1.tif', 'images/California/white_2.tif', 
         'images/Indonesia/white_1.tif', 'images/Indonesia/white_2.tif']
arrays = [Image_processor(path=p).imgs['original'] for p in paths]
comb_image = np.vstack([np.hstack(arrays[:2]), np.hstack(arrays[2:])])
IP_combo = Image_processor(img=comb_image)
IP_combo.plot()

### SLIC time analysis

In [0]:
s = 400 # image size
img = comb_image[:s, :s]
g = 100
bin_grid = [g, g]

In [0]:
%%timeit
obj = SLIC(img, bin_grid)
obj.iterate(10)

**Times varying grid size**

size 400 used

(2, 2) 1 loop, best of 3: 4.87 s per loop

(4, 4) 1 loop, best of 3: 7.49 s per loop

(5, 5) 1 loop, best of 3: 8.16 s per loop

(8, 8) 1 loop, best of 3: 9.47 s per loop

(10, 10) 1 loop, best of 3: 10.1 s per loop

(16, 16) 1 loop, best of 3: 12.3 s per loop

(20, 20) 1 loop, best of 3: 14 s per loop

(25, 25) 1 loop, best of 3: 16.5 s per loop

(40, 40) 1 loop, best of 3: 25.8 s per loop

(50, 50) 1 loop, best of 3: 34.3 s per loop

(100, 100) 1 loop, best of 3: 1min 45s per loop

In [0]:
times = [4.87, 7.49, 8.16, 9.47, 10.1, 12.3, 14, 16.5, 25.8, 34.3, 60+45]
sizes = [2, 4, 5, 8, 10, 16, 20, 25, 40, 50, 100]
centroids = np.array(sizes)**2

plt.figure(figsize=[8,8])
plt.loglog(centroids, times, 'bo-', label='Times')


x = np.logspace(1, 4, 10)
y = .5 * x**0.5
plt.loglog(x, y, 'g--', label=r'$\mathcal{O}(10^\frac{1}{2})$)')


plt.gca().set(title='SLIC Grid Size Runtime', xlabel='Number of Grid Points',
              ylabel='Runtime [s]')
plt.legend(loc='upper left', fontsize=14)

plt.savefig('SLIC_grid_runtimes.png')

**Times varying grid size**

size 500 used

(5,5) 1 loop, best of 3: 12.8 s per loop

(10, 10) 1 loop, best of 3: 15.7 s per loop

(20, 20) 1 loop, best of 3: 21 s per loop

**Times varying image size**

grid (10, 10) used

s = 50, 1 loop, best of 3: 390 ms per loop

s = 100, 1 loop, best of 3: 884 ms per loop

s = 150, 1 loop, best of 3: 1.75 s per loop

s = 200, 1 loop, best of 3: 2.86 s per loop

s = 300, 1 loop, best of 3: 5.97 s per loop

s = 400, 1 loop, best of 3: 10.2 s per loop

(without GPU 1 loop, best of 3: 10.4 s per loop)

s = 600, 1 loop, best of 3: 22.5 s per loop

s = 800, 1 loop, best of 3: 39.9 s per loop

s = 1000, 1 loop, best of 3: 1min 2s per loop

s = 1200, 1 loop, best of 3: 1min 29s per loop

s = 1600, 1 loop, best of 3: 2min 37s per loop

Runtime with GPU: 10.2 s

Runtime without GPU: 10.4s

In [0]:
times = [390e-3, 884e-3, 1.75, 2.86, 5.97, 10.2, 22.5, 39.9, 60 + 2, 60 + 29, 120 + 37]
sizes = [50, 100, 150, 200, 300, 400, 600, 800, 1000, 1200, 1600]
pixels = np.array(sizes)**2

plt.figure(figsize=[8,8])
plt.loglog(pixels, times, 'bo-', label='Times')

x = np.logspace(4, 7, 10)
y = 1e-5 * x**1
plt.loglog(x, y, 'r--', label=r'$\mathcal{O}(10^1)$)')


plt.gca().set(title='SLIC Image Size Runtime', xlabel='Number of Pixels',
              ylabel='Runtime [s]')
plt.legend(loc='upper left', fontsize=14)

plt.savefig('SLIC_size_runtimes.png')

### SLIC memory analysis

In [0]:
!pip install memory_profiler

In [0]:
%load_ext memory_profiler

In [0]:
%memit [range(0, i) for i in range(1, 100)]

In [0]:
%%memit

s = 1200 # image size
img = comb_image[:s, :s]
bin_grid = [10, 10]

obj = SLIC(img, bin_grid)
obj.iterate(10)

s = 50,

s = 100,

s = 400, peak memory: 811.61 MiB, increment: 0.08 MiB

s = 1600, peak memory: 1022.53 MiB, increment: 0.01 MiB

# Butterfly


## over segmentation

In [0]:
butterfly_IP = Image_processor(path='images/butterfly.tif')
butterfly_IP.plot()

In [0]:
# blur the image to reduce spotting effect
butterfly_IP.reset()
butterfly_IP.gauss(sigma=3, key='blur')

# create the SLIC object iterate it and plot
butterfly_SLIC = SLIC(butterfly_IP.imgs['blur'], [25, 25], dist_metric_args=[1.])
butterfly_SLIC.iterate(10)
butterfly_SLIC.plot()

## Create Segments and Define Extraction Functions

In [0]:
# extract mask
butterfly_mask = butterfly_SLIC.get_segmentation()

# create segment groups, enforce size and plot
butterfly_segs = segment_group(butterfly_mask)
butterfly_segs.enforce_size(min_size=50)

# plot the segments
butterfly_segs.plot(back_img=butterfly_IP.imgs['original'])

In [0]:
# obtain the edge detected image
butterfly_IP.reset()
butterfly_IP.scharr()
butterfly_IP.grey_scale()
butterfly_IP.threshold(0.05, key='scharr')
butterfly_IP.plot('scharr')

In [0]:
# define edge extraction function
def edge_extraction(Xs, Ys, scharr_img):
    return scharr_img[Ys, Xs].mean() / scharr_img.std()

# define color extracton function
def color_extraction(Xs, Ys, img1):
    """
    Takes in a single image and returns the average at the cordinates in
    each color channel
    """
    avgs1 = img1[Ys, Xs].mean(axis=0)
    return [*avgs1]

In [0]:
# older extraction functions


# extract color and texture features
# titles = ['red_avg', 'green_avg', 'blue_avg', 'lap_avg']
# def extraction(Xs, Ys, img_col, img_grad):
#     """
#     """
#     avgs_col = img_col[Ys, Xs].mean(axis=0)
#     avg_grad = (img_grad[Ys, Xs]**2).mean()
#     return [*avgs_col, avg_grad]


# extract color and std features
# titles = ['red_avg', 'green_avg', 'blue_avg',
#           'red_std', 'green_std', 'blue_std']
# def extraction(Xs, Ys, img_col):
#     """
#     """
#     avgs_col = img_col[Ys, Xs].mean(axis=0)
#     stds_col = img_col[Ys, Xs].std(axis=0)
#     return [*avgs_col, *stds_col*5]


# extract color and gabor features
# titles = ['red_avg', 'green_avg', 'blue_avg',
#           'gabor_avg', 'gabor_std']
# def extraction(Xs, Ys, img_col, img_gab):
#     """
#     """
#     avgs_col = img_col[Ys, Xs].mean(axis=0)
#     avgs_gab = (img_gab[Ys, Xs]**2).mean()
#     stds_gab = img_gab[Ys, Xs].std()
#     return [*avgs_col, avgs_gab*3, stds_gab]


## Merge by Clustering

In [0]:
# extract features
butterfly_feats = butterfly_segs.feature_extraction(color_extraction,
                                                    [butterfly_IP.imgs['blur']])

# inspect features
pd.DataFrame(butterfly_feats, columns=['red_avg', 'green_avg', 'blue_avg']).describe()

In [0]:
# normalise features if need be
# butterfly_feats = butterfly_feats - butterfly_feats.mean(axis=0)
# butterfly_feats = butterfly_feats / butterfly_feats.std(axis=0)

# inspect features
# pd.DataFrame(butterfly_feats, columns=titles).describe()

In [0]:
# do an AGNES clustering with these features
butterly_AGNES = AGNES(butterfly_feats)
butterly_AGNES.iterate()
butterly_AGNES.cluster_distance_plot('all')

In [0]:
# get the clustering and assign it
butterly_clusters = butterly_AGNES.cluster_by_derivative(n_std=2., plot=False)
butterfly_segs.assign_clusters(butterly_clusters)

# plot these clusters
butterfly_segs.plot('cluster_all', back_img=butterfly_IP.imgs['original'])

In [0]:
# implement cluster merging (no edge presence used)
butterfly_segs.merge_by_cluster()
butterfly_segs.plot('merged_edges', back_img=butterfly_IP.imgs['original'])

## Merging by Edge Confidence

In [0]:
# Identify edge confs
butterfly_segs.edge_confidence(edge_extraction, [butterfly_IP.imgs['scharr']])

# plot them
butterfly_segs.plot('edge_conf', back_img=butterfly_IP.imgs['original'])

In [0]:
# merge
butterfly_segs.merge_by_edge(edge_absent=.2)
butterfly_segs.plot('merged_edges', back_img=butterfly_IP.imgs['original'])

## repeate merging process again (if wanted)

In [0]:
# repeat the process for the new segmentation

# feature extraction
butterfly_feats = butterfly_segs.feature_extraction(extraction,
                                                    [butterfly_IP.imgs['blur']])

# clustering
butterly_AGNES = AGNES(butterfly_feats)
butterly_AGNES.iterate()

# get the clustering and assign it
butterly_clusters = butterly_AGNES.cluster_by_derivative(n_std=2., plot=False)
butterfly_segs.assign_clusters(butterly_clusters)

In [0]:
# plot these clusters
butterfly_segs.plot('cluster_all', back_img=butterfly_IP.imgs['original'])

In [0]:
# implement cluster merging (no edge presence used)
butterfly_segs.merge_by_cluster()
butterfly_IP.plot('original')
butterfly_segs.plot('merged_edges', ax=plt.gca())

## Segmentation Analysis

In [0]:
# extract the cluster mask
butterfly_cluster = butterfly_segs.get_cluster_mask()
plt.imshow(butterfly_cluster)

# extract the segments mask
butterfly_mask = butterfly_segs.mask

In [0]:
# butterfly_cluster = butterfly_SA.clusters
butterfly_SA = Segment_Analyser(butterfly_IP.imgs['original'],
                      butterfly_mask,
                      butterfly_cluster)
butterfly_SA.set_labels()

In [0]:
butterfly_SA.plot_clusters()

In [0]:
butterfly_SA.get_composition()
print('\n')
butterfly_SA.get_grain_count()

In [0]:
butterfly_SA.get_gsd('wing')

In [0]:
butterfly_SA.get_span('wing')

# Thin Sections
This is the sequence used for each TS image, all that was changed was the load data section.

## Texture, color cluster comparison

### Load Images

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])

# load data
IP_polar = Image_processor(path='images/Indonesia/polar_3.tif')
IP_white = Image_processor(path='images/Indonesia/white_3.tif')

# plot them
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    IP.plot(ax=ax)
    ax.axis('off')

# plt.savefig('images/Results/SS_both.tif')

### over segmentation

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])

# guassian blur for color space
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    IP.reset()
    IP.gauss(sigma=3, key='gauss')
    IP.plot('gauss', ax=ax)
    ax.axis('off')
    
# plt.savefig('image/Results/SS_both_blured.tif')

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
[ax.axis('off') for ax in axs.ravel()]

MSLIC_obj = MSLIC_wrapper([IP_polar.imgs['original'],
                           IP_white.imgs['original']],
                          bin_grid=[25, 25],
                          dist_metric_args=[.5])
MSLIC_obj.iterate(10)
MSLIC_obj.plot(axs=axs.ravel())

### create segment object

In [0]:
# extract the mask
mask = MSLIC_obj.get_segmentation()

# initalise segement object and remove small grains
segment_obj = segment_group(mask)

# plot the segmentations
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    segment_obj.plot(option='default', back_img=IP.imgs['original'], ax=ax)
    ax.axis('off')
    
# plt.savefig()

In [0]:
# remove small grains
segment_obj.enforce_size(min_size=50)

### color based clustering

In [0]:
# define extraction function (extract only color features)
titles = ['red_nonpolar', 'green_nonpolar', 'blue_nonpolar',
          'red_polar', 'green_polar', 'blue_polar']
def extraction(Xs, Ys, img1, img2):
    """
    Takes in two images and returns the average at the cordinates in
    each color channel for both images
    """
    avgs1 = img1[Ys, Xs].mean(axis=0)
    avgs2 = img2[Ys, Xs].mean(axis=0)
    return [*avgs1, *avgs2]

# extract features
feats = segment_obj.feature_extraction(extract_func = extraction,
                                       func_vars = [IP_white.imgs['gauss'],
                                                    IP_polar.imgs['gauss']])
pd.DataFrame(feats, columns=titles).describe()

In [0]:
# cluster by AGNES
AGNES_obj = AGNES(feats)
AGNES_obj.iterate()
AGNES_obj.cluster_distance_plot('2nd')

In [0]:
# form the clustering
clustering = AGNES_obj.cluster_by_derivative(n_std=3., plot=False)
segment_obj.assign_clusters(clustering)

# plot all clusters
segment_obj.plot('cluster_all', back_img=IP_white.imgs['original'])

# plt.savefig('Gabon_1_clusterings.tif')

### Texture based clustering

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    IP.reset()
    IP.laplace(key='lap')
    IP.plot(key='lap', ax=ax)

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    IP.reset()
    IP.grey_scale()
    IP.laplace(key='lap')
    IP.plot(key='lap', ax=ax)

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    IP.reset()
    IP.grey_scale()
    IP.gabor_filters(2, 5, key='gab')
    IP.plot(key='gab', ax=ax)

In [0]:
# define extraction function (extract only color features)
# titles = ['lap_avg_nonpolar', 'lap_std_nonpolar',
#           'lap_avg_polar', 'lap_std_polar']

def extraction(Xs, Ys, img1, img2):
    """
    Takes in two images and returns the average at the cordinates in
    each color channel for both images
    """
    avg1 = img1[Ys, Xs].mean(axis=0)
    avg2 = img2[Ys, Xs].mean(axis=0)
    return [*avg1, *avg2]



# titles = ['red_std_nonpolar', 'green_std_nonpolar', 'blue_std_nonpolar',
#           'red_std_polar', 'green_std_polar', 'blue_std_polar']
def extraction_std(Xs, Ys, img1, img2):
    std1 = img1[Ys, Xs].std(axis=0)
    std2 = img2[Ys, Xs].std(axis=0)
    return [*std1, *std2]


titles= ['gab_avg_nonpolar', 'gab_std_nonpolar',
         'gab_avg_polar', 'gab_std_polar']
def extraction_single(Xs, Ys, img1, img2):
    avg1 = img1[Ys, Xs].mean()
    avg2 = img2[Ys, Xs].mean()
    std1 = img1[Ys, Xs].std()
    std2 = img2[Ys, Xs].std()
    return [avg1, avg2, std1, std2]

# extract features
feats = segment_obj.feature_extraction(extract_func = extraction_single,
                                       func_vars = [IP_white.imgs['lap'],
                                                    IP_polar.imgs['lap']])
pd.DataFrame(feats, columns=titles).describe()

In [0]:
# cluster by AGNES
AGNES_obj = AGNES(feats)
AGNES_obj.iterate()
AGNES_obj.cluster_distance_plot('2nd')

In [0]:
# form the clustering
clustering = AGNES_obj.cluster_by_derivative(n_std=3., plot=False)
segment_obj.assign_clusters(clustering)

# plot all clusters
segment_obj.plot('cluster_all', back_img=IP_white.imgs['original'])

# plt.savefig('Gabon_1_clusterings.tif')

## Porosity determination

### Load Images

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])

# load data
IP_polar = Image_processor(path='images/Thailand/polar_1.tif')
IP_white = Image_processor(path='images/Thailand/white_1.tif')

# plot them
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    IP.plot(ax=ax)
    ax.axis('off')

plt.savefig('proc_1.png')

### over segmentation

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])

# guassian blur for color space
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    IP.reset()
    IP.gauss(sigma=3, key='gauss')
    IP.plot('gauss', ax=ax)
    ax.axis('off')
    
plt.savefig('proc_2.png')

In [0]:
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
[ax.axis('off') for ax in axs.ravel()]

MSLIC_obj = MSLIC_wrapper([IP_polar.imgs['gauss'],
                           IP_white.imgs['gauss']],
                          bin_grid=[25, 25],
                          dist_metric_args=[.5])
MSLIC_obj.iterate(10)
MSLIC_obj.plot(axs=axs.ravel())

plt.savefig('proc_3.png')

### segment clustering

In [0]:
# extract the mask
mask = MSLIC_obj.get_segmentation()

# initalise segement object and remove small grains
segment_obj = segment_group(mask)

# plot the segmentations
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    segment_obj.plot(option='default', back_img=IP.imgs['original'], ax=ax)
    ax.axis('off')
    
plt.savefig('proc_4.png')

In [0]:
# remove small grains
segment_obj.enforce_size(min_size=50)

In [0]:
# define extraction function (extract only color features)
titles = ['red_nonpolar', 'green_nonpolar', 'blue_nonpolar',
          'red_polar', 'green_polar', 'blue_polar']
def extraction(Xs, Ys, img1, img2):
    """
    Takes in two images and returns the average at the cordinates in
    each color channel for both images
    """
    avgs1 = img1[Ys, Xs].mean(axis=0)
    avgs2 = img2[Ys, Xs].mean(axis=0)
    return [*avgs1, *avgs2]

# extract features
feats = segment_obj.feature_extraction(extract_func = extraction,
                                       func_vars = [IP_white.imgs['gauss'],
                                                    IP_polar.imgs['gauss']])
pd.DataFrame(feats, columns=titles).describe()

In [0]:
# cluster by AGNES
AGNES_obj = AGNES(feats)
AGNES_obj.iterate()
AGNES_obj.cluster_distance_plot('2nd')

In [0]:
# form the clustering
clustering = AGNES_obj.cluster_by_derivative(n_std=3., plot=False)
segment_obj.assign_clusters(clustering)

# plot all clusters
segment_obj.plot('cluster_all', back_img=IP_white.imgs['original'])

plt.savefig('proc_5.png')

### edge detection

In [0]:
# edge detected image
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    IP.reset()
    IP.grey_scale()
    IP.canny()
    IP.gauss_grey(sigma=3)
    IP.threshold(.15, 'edges')
    IP.plot('edges', ax=ax)
    ax.axis('off')
    
plt.savefig('proc_6.png')

In [0]:
# define the edge confidence funciton
def edge_conf(Xs, Ys, img1, img2):
    return img1[Ys, Xs].mean() + img2[Ys, Xs].mean()

# calculate edge condifences
segment_obj.edge_confidence(edge_conf, [IP_polar.imgs['edges'],
                                        IP_white.imgs['edges']])

# plot them
segment_obj.plot('edge_conf', back_img=IP_white.imgs['original'])

plt.savefig('proc_7.png')

### Segment Merging

In [0]:
# merge connected segments
segment_obj.merge_by_cluster(edge_present=1.)
# segment_obj.merge_by_edge(edge_absent=.25)

# plot the results
fig, axs = plt.subplots(1, 2, figsize=[20, 10])
for IP, ax in zip([IP_white, IP_polar], axs.ravel()):
    segment_obj.plot('merged_edges', back_img=IP.imgs['original'], ax=ax)
    ax.axis('off')
#     ax.set(title='Post Cluster Merging')
    
plt.savefig('proc_8.png')

### segment analysis

In [0]:
# extract mask and clustering
seg_mask = segment_obj.mask
clust_mask = segment_obj.get_cluster_mask()

# tx2_clusters = SA_obj.clusters # keep already clusters labels
SA_obj = Segment_Analyser(IP_white.imgs['original'],
                          seg_mask,
                          clust_mask)
SA_obj.set_labels()

In [0]:
SA_obj.plot_clusters()

plt.savefig('proc_9.png')

SA_obj.get_composition()

In [0]:
SA_obj.get_gsd('rock')
plt.savefig('proc_10.png')

### Save Segementations

In [0]:
# save this mask
# np.savetxt('masks/TX2_MSLIC_25_25.tif', MSLIC_obj.get_segmentation())

In [0]:
# save the masks
# np.savetxt('/content/masks/both_30_30_merged_6_channel_color.txt', segments.mask)
# np.save('/content/masks/both_30_30_merged_6_channel_color_groupings.npy', segments.seg_clusters)

### Save Results

Indonesia 1
Tabel of Compositions
0.47 %	 silt
66.52 %	 rock
33.01 %	 pore

Indonesia 2
Tabel of Compositions
63.77 %	 rock
25.94 %	 pore
10.3 %	 silt

Indonesia 3
Tabel of Compositions
26.29 %	 pore
14.83 %	 silt
58.88 %	 rock

Gabon 1
Tabel of Compositions
11.92 %	 pore
39.76 %	 silt
32.54 %	 rock
15.78 %	 silt/pore

Gabon 2
Tabel of Compositions
48.52 %	 silt
13.91 %	 silt/pore
37.57 %	 rock

Gabon 3
Tabel of Compositions
55.89 %	 silt
30.03 %	 rock
14.08 %	 pore

Norway 1
Tabel of Compositions
13.16 %	 pore
12.26 %	 silt
74.58 %	 rock

Norway 2
Tabel of Compositions
14.23 %	 pore
29.21 %	 silt
39.97 %	 rock
16.59 %	 silt/pore

Norway 3
Tabel of Compositions
44.19 %	 rock
15.03 %	 pore
30.77 %	 silt

Texas 2
Tabel of Compositions
19.36 %	 pore
80.64 %	 rock

Texas 3
Tabel of Compositions
13.84 %	 pore
9.57 %	 silt
63.67 %	 rock
12.91 %	 silt/pore

Texas 4
Tabel of Compositions
14.96 %	 pore
29.0 %	 silt
56.05 %	 rock

Thailand 1
Tabel of Compositions
15.54 %	 pore
72.69 %	 rock
11.77 %	 silt

Thailand 2
Tabel of Compositions
65.8 %	 rock
17.27 %	 silt
16.93 %	 pore

Thailand 3
Tabel of Compositions
26.05 %	 pore
66.76 %	 rock
7.20 %	 silt

California 1
Tabel of Compositions
28.54 %	 silt
4.97 %	 pore
66.49 %	 rock

California 2
Tabel of Compositions
11.98 %	 silt
75.48 %	 rock
12.54 %	 pore

California 3
Tabel of Compositions
15.19 %	 pore
71.5 %	 rock
13.31 %	 silt

In [0]:
titles = ['Indonesia', 'Gabon   ', 'Norway   ', 'Texas   ', 'Thailand', 'California']
results = np.array([[33.01, 25.94, 26.29],
                    [11.92, 13.91, 14.08],
                    [13.16, 14.23, 15.03],
                    [19.36, 13.84, 14.96],
                    [15.54, 16.93, 26.05],
                    [4.97, 12.54, 15.19]])

actual = np.array([22.0, 20.7, 21.9, 21.0, 19.6, 15.1])

print('Region \t\t Avg \t\t Std \t\t Actual \t Diff')
print('-'*72)
for l, r, a in zip(titles, results, actual):
    avg = round(r.mean(),2)
    std = round(r.std(),2)
    print(l, '\t', avg, '\t\t', std, '\t\t', a, '\t\t', round((avg-a),2))